In [1]:
import random

COLORS = ["blue", "yellow", "red", "black", "white"]
COUNT = 20

In [2]:
class Bag:
    def __init__(self):
        self.tiles = COLORS * COUNT
        self.discard_tiles = []

    def draw_tile(self):
        if len(self.tiles) == 0:
            if len(self.discard_tiles) == 0:
                return None
            else:
                self.refill_bag()
        
        random_index = random.randint(0, len(self.tiles) - 1)
        drawn_tile = self.tiles.pop(random_index)
        return drawn_tile
    
    def refill_bag(self):
        self.tiles.extend(self.discard_tiles)
        self.discard_tiles = []

In [3]:
class Factory:
    def __init__(self):
        self.tiles = []

    def add_tiles(self, tiles_list):
        self.tiles.extend(tiles_list)

    def remove_tiles_by_color(self, color):
        self.tiles = [tile for tile in self.tiles if tile != color]

In [4]:
class Player:
    def __init__(self, player_name):
        self.player_name = player_name
        self.pattern_lines = [[],[],[],[],[]]
        self.wall = {color: [] for color in COLORS}
        self.floor_line = []
        self.score = 0
        
#     def place_tiles_in_pattern_line(self, line_index, tiles):
#         line_index -= 1
        
#         if line_index not in [0, 1, 2, 3, 4, 5]:
#             print("Invalid line index.")
#             return
    
#         if line_index == 5:
#             self.floor_line.extend(tiles)
            
#         else:
#             selected_line = self.pattern_lines[line_index]
#             if len(selected_line) == (line_index+1):
#                 print("Selected line is full.")
#                 return
            
#             if (len(selected_line) > 0) and (selected_line[0] != tiles[0]):
#                 print("Selected line has unmatched color.")
#                 return 
            
#             if "1" in tiles:
#                 self.floor_line.append("1")
#                 tiles.pop()
                
#             result_line = selected_line + tiles
#             self.pattern_lines[line_index] = result_line[0:line_index+1]
#             self.floor_line.extend(result_line[line_index+1:])
            
    def display_score(self):
        print(f"{self.player_name}'s Score: {self.score}")

    def display_board(self):
        print(f"{self.player_name}'s Board:")
        print(self.pattern_lines)
        print(self.floor_line)

In [5]:
class Game:
    def __init__(self, players):
        self.num_players = len(players)
        self.players = players
        
        self.bag = Bag()
        self.factory_count = self.num_players * 2 + 1
        self.factories = [Factory() for _ in range(self.factory_count)]
        self.center_display = Factory()
        
        self.first_player_token = True
        self.current_player = 0

    def next_player(self):
        if self.current_player == (self.num_players - 1):
            self.current_player = 0
        else:
            self.current_player += 1
    
    def initialize_factories(self):
        for _ in range(4):
            for factory in self.factories:
                drawn_tile = self.bag.draw_tile()
                if drawn_tile:
                    factory.add_tiles([drawn_tile])
    
    def display_factories(self):
        print("Current factories:")
        for factory in self.factories:
            print(factory.tiles)
        print("Current Center:")
        print(self.center_display.tiles)
        
    def num_remaining_tiles_in_factory(self):
        return sum([len(factory.tiles) for factory in self.factories]) + len(self.center_display.tiles)
        
    def take_tiles_from_factory(self, factory_index, color, verbose=False):
        factory_index -= 1
        
        # Check if the factory index is valid
        if factory_index < 0 or factory_index > self.factory_count:
            if verbose:
                print("Invalid factory index.")
            return 0
                   
        if factory_index == self.factory_count:
            selected_factory = self.center_display
        else:
            selected_factory = self.factories[factory_index]

        # Check if the selected factory is empty
        if not selected_factory.tiles:
            if verbose:
                print("Selected factory is empty.")
            return 0

        selected_tiles = [tile for tile in selected_factory.tiles if tile == color]

        # Check if there are tiles of the selected color in the factory
        if not selected_tiles:
            if verbose:
                print(f"No {color} tiles in the selected factory.")
            return 0
        
        if factory_index == self.factory_count:
            self.center_display.remove_tiles_by_color(color)
            if self.first_player_token:
                self.first_player_token = False
                selected_tiles.append("1")
        else:
            unselected_tiles = [tile for tile in selected_factory.tiles if tile != color]
            self.factories[factory_index].tiles = []
            self.center_display.add_tiles(unselected_tiles)
            
        return selected_tiles
    
    def place_tiles_in_pattern_line(self, line_index, tiles, verbose=False):
        line_index -= 1
        
        if line_index not in [0, 1, 2, 3, 4, 5]:
            if verbose:
                print("Invalid line index.")
            return 0
    
        if line_index == 5:
            self.players[self.current_player].floor_line.extend(tiles)
            return 1
            
        else:
            selected_line = self.players[self.current_player].pattern_lines[line_index]
            if len(selected_line) == (line_index+1):
                if verbose:
                    print("Selected line is full.")
                return 0
            
            if (len(selected_line) > 0) and (selected_line[0] != tiles[0]):
                if verbose:
                    print("Selected line has unmatched color.")
                return 0
            
            if "1" in tiles:
                self.players[self.current_player].floor_line.append("1")
                tiles.pop()
                
            result_line = selected_line + tiles
            self.players[self.current_player].pattern_lines[line_index] = result_line[0:line_index+1]
            self.players[self.current_player].floor_line.extend(result_line[line_index+1:])
            return 1
        
    def random_player_turn(self, verbose=True):
        tiles = 0
        while not tiles:
            random_factory_index = random.randint(1, g.factory_count+1)
            random_color = random.choice(COLORS)
            tiles = self.take_tiles_from_factory(random_factory_index, random_color)

        placed = 0
        while not placed:
            random_line_index = random.randint(1, 6)
            placed = self.place_tiles_in_pattern_line(random_line_index, tiles)
        
        if verbose:
            print(f"{g.players[g.current_player].player_name} takes {len(tiles)} {tiles[0]} tiles from factory {random_factory_index}.")
            print(f"{self.players[g.current_player].player_name} places {len(tiles)} {tiles[0]} tiles to line {random_line_index}")
            self.players[self.current_player].display_board()
            self.display_factories()

    def player_turn(self, factory_index, color, line_index, verbose=True):
        tiles = self.take_tiles_from_factory(random_factory_index, random_color)
        placed = self.place_tiles_in_pattern_line(random_line_index, tiles)
        if verbose:
            print(f"{g.players[g.current_player].player_name} takes {len(tiles)} {tiles[0]} tiles from factory {random_factory_index}.")
            print(f"{self.players[g.current_player].player_name} places {len(tiles)} {tiles[0]} tiles to line {random_line_index}")
            self.players[self.current_player].display_board()
            self.display_factories()
    
    def play(self):
        pass
#         while [len(x) for x in self.]

In [6]:
player1 = Player("Random Player 1")
player2 = Player("Random Player 2")
g = Game([player1, player2])

In [7]:
g.initialize_factories()

In [8]:
g.display_factories()

Current factories:
['black', 'black', 'yellow', 'white']
['white', 'white', 'white', 'red']
['yellow', 'red', 'yellow', 'red']
['white', 'black', 'red', 'red']
['blue', 'blue', 'yellow', 'yellow']
Current Center:
[]


In [9]:
while g.num_remaining_tiles_in_factory():
    g.random_player_turn()
    g.next_player()

Random Player 1 takes 1 white tiles from factory 1.
Random Player 1 places 1 white tiles to line 3
Random Player 1's Board:
[[], [], ['white'], [], []]
[]
Current factories:
[]
['white', 'white', 'white', 'red']
['yellow', 'red', 'yellow', 'red']
['white', 'black', 'red', 'red']
['blue', 'blue', 'yellow', 'yellow']
Current Center:
['black', 'black', 'yellow']
Random Player 2 takes 2 blue tiles from factory 5.
Random Player 2 places 2 blue tiles to line 5
Random Player 2's Board:
[[], [], [], [], ['blue', 'blue']]
[]
Current factories:
[]
['white', 'white', 'white', 'red']
['yellow', 'red', 'yellow', 'red']
['white', 'black', 'red', 'red']
[]
Current Center:
['black', 'black', 'yellow', 'yellow', 'yellow']
Random Player 1 takes 1 red tiles from factory 2.
Random Player 1 places 1 red tiles to line 5
Random Player 1's Board:
[[], [], ['white'], [], ['red']]
[]
Current factories:
[]
[]
['yellow', 'red', 'yellow', 'red']
['white', 'black', 'red', 'red']
[]
Current Center:
['black', 'black'